##Developing to the Facebook API##

## The Steps

Note that only the most important steps of the OAuth Dance will be mentioned here. Some will be omitted

### Preliminary Step: Become a Registered Developer and Create an App

For Facebook, as for any other enviroment with the separation of the resource owner from the app/client/consumer, the provider must be able to **identify/authenticate** the app/client/consumer.  This involves the human developer creating an account and further defining a particular App.  The same developer may, in fact, create multiple distinct Apps, but for now we will assume a single App.

In the case of Facebook, we first logged in as our normal Facebook user.  We then followed the directions as posted under Facebook Developer:

https://developers.facebook.com/docs/apps/register

In [3]:
import json
import requests

Reading the contents of `creds.json` into a Python dictionary data structure. This file is a place holder for all of my credentials (Trang)

In [4]:
with open("creds.json", "r") as file:
    creds = json.load(file)
print(str(creds['facebook']))

{'appid': '1532167366905914', 'appsecret': '8cfdcfe4fb92e6af2119a6eff9325a36', 'redirect_uri': 'https://localhost/callback', 'code': 'AQAwGgQYRIle3vVR7OjrzMHmdhEiJCDpdrROlfE7tHXZxZGFIdQGYAhc5tKk7qamDq6T7TeH2O7j-hptrbCnKkMK_pUhGseXHIthVbwmDsl9whPscUZXSO6c8SXZUqeTWa-ST2_TN3_RIZzjxUAPhuVSVD5alRxlHmeICuA-k9K03BM6VUBaq-bpt8apccqkR5Bm3fIerG9y3zUZPWHcZrasvbqPj-QlRG8I239byGxy3UxKKHR9RlJLuJeE6ydTmWA80XFF52rWlUOK5TBNS6T3cjnrLTKarkiM-xbecLLQGwRsyFN3WNe4k-CiK318TZWxCrUkPrI3tZyAnl7VehrF', 'state': '1234567890', 'scope': ['public_profile', 'user_likes', 'user_photos', 'user_posts'], 'mytoken': 'EAAVxf4OkEDoBAE4cThIm0acjgpGCId0dPzT5krPF8o2LAZC627lsr6EZAaMQrcQbe4hPmYiycUFkzy9N3La4MGl0ImmdDHC1YiQ9q04OZCL9ZC9UvI9SoMLNMtGzK5cAzOCaVBVkv6bjnZCjybd5G1f126s3veJf76rmZAELCjBwZDZD'}


### Step 1. Construct the URL to give to the User/Resource Owner

> **Line 1** of the OAuth Dance

For Facebook, we are using the documentation at

https://developers.facebook.com/docs/facebook-login/access-tokens

Through the documentation, we are able to determine the protocol, location, resource, and parameters to be used in the constructed URL.  In particular, for Facebook, we have:

variable | value | comment
---------|-------|:--------
protocol | `https` | Given in GET specification
location | `www.facebook.com` | Given in GET specification
resource  | `/v2.12/dialog/oauth?` | Given in GET specification
client_id | `creds['facebook']['appid']` | Based on App registration and entering info in `creds.json`
redirect_uri | `creds['facebook']['redirect_uri]` | Based on App registration and entering info in `creds.json`
scopes | `creds['facebook']['scope']` | Possible values specified at https://developers.facebook.com/docs/facebook-login/permissions.

We want to look at the following scopes `public_profile`, `user_likes`, `user_photos`, `user_posts`, which have been specified in the `creds.json` file.

We construct the base of the `url` the same way we did before

In [11]:
def buildBaseURL(location, auth_resource, protocol='https'):
    '''
    This function takes 3 parameters: a default value of protocol, a location, and a resource to 
    construct a complete base URL. In this case, protocol is optional
    
    Return: A string URL
    '''
    template = "{}://{}{}"
    authurl = template.format(protocol, location, auth_resource)
    return authurl

In [8]:
location = "www.facebook.com"
auth_resource = "/v2.12/dialog/oauth?"
authurl = buildBaseURL(location, auth_resource, protocol='https')
print(authurl)

https://www.facebook.com/v2.12/dialog/oauth?


In [22]:
#Load the parameters specified in the creds.json into a dictionary
urlquery = {}
urlquery['client_id'] = creds['facebook']['appid']
urlquery['redirect_uri'] = creds['facebook']['redirect_uri']
urlquery['response_type'] = 'code'
urlquery['state'] = creds['facebook']['state']
urlquery['scope'] = creds['facebook']['scope']
urlquery

{'client_id': '1532167366905914',
 'redirect_uri': 'https://localhost/callback',
 'response_type': 'code',
 'scope': ['public_profile', 'user_likes', 'user_photos', 'user_posts'],
 'state': '1234567890'}

In [20]:
def buildURLwithParams(location, auth_resource, pdict):
    '''
    This function takes three parameters: a location, a resource and a dictionary, 
    construct an authorization URL
    
    Return: A string URL
    '''
    
    authurl = buildBaseURL(location, auth_resource, protocol='https')
    session = requests.Session() 
    p = requests.Request('GET', authurl, params=pdict).prepare()
    return p.url

In [21]:
location = "www.facebook.com"
auth_resource = "/v2.12/dialog/oauth?"
auth_url = buildURLwithParams(location, auth_resource, urlquery)
print(auth_url)

https://www.facebook.com/v2.12/dialog/oauth?client_id=1532167366905914&redirect_uri=https%3A%2F%2Flocalhost%2Fcallback&response_type=code&state=1234567890&scope=public_profile&scope=user_likes&scope=user_photos&scope=user_posts


### Step 2. User Agent/Resource Owner Interacts with Authorization Sever

Using the above link, the user (myself) will authorize my app named `FinalProject` to read my own data. The following authorization window will pop up

<img src="Authorization.png" width="500">

### Step 3. Authorization Server Redirects User's Web Browser to App/Client

Once I click `Continue as Trang` which means I authorized the app to read my data, I was redirected to this site, where I can obtain the `code` which is hidden in the `url`

<img src="LocalHost.png" width="700">

### Step 4. User/Resource Owner Communcates `code` and `state` to the App

Once we obtained the `code` and `app`, we will update the `creds.json` file


In [25]:
with open("creds.json", "r") as file:
    creds = json.load(file)
print(str(creds['facebook']))

{'appid': '1532167366905914', 'appsecret': '8cfdcfe4fb92e6af2119a6eff9325a36', 'redirect_uri': 'https://localhost/callback', 'code': 'AQAwGgQYRIle3vVR7OjrzMHmdhEiJCDpdrROlfE7tHXZxZGFIdQGYAhc5tKk7qamDq6T7TeH2O7j-hptrbCnKkMK_pUhGseXHIthVbwmDsl9whPscUZXSO6c8SXZUqeTWa-ST2_TN3_RIZzjxUAPhuVSVD5alRxlHmeICuA-k9K03BM6VUBaq-bpt8apccqkR5Bm3fIerG9y3zUZPWHcZrasvbqPj-QlRG8I239byGxy3UxKKHR9RlJLuJeE6ydTmWA80XFF52rWlUOK5TBNS6T3cjnrLTKarkiM-xbecLLQGwRsyFN3WNe4k-CiK318TZWxCrUkPrI3tZyAnl7VehrF', 'state': '1234567890', 'scope': ['public_profile', 'user_likes', 'user_photos', 'user_posts'], 'mytoken': 'EAAVxf4OkEDoBAE4cThIm0acjgpGCId0dPzT5krPF8o2LAZC627lsr6EZAaMQrcQbe4hPmYiycUFkzy9N3La4MGl0ImmdDHC1YiQ9q04OZCL9ZC9UvI9SoMLNMtGzK5cAzOCaVBVkv6bjnZCjybd5G1f126s3veJf76rmZAELCjBwZDZD'}


### Step 5. App Exchanges Code for a Token

> **Line 7** and **Line 8** of the OAuth Dance

Once we obtained a `code` from the previous step, we will exchange this `code` for an `access token`.
This is again a time for gathering information from the Provider on the information necessary to make the appropriate GitHub request to get this done.  We must determine the HTTP method, protocol, location, resource, and parameters to be used.  In particular, for Facebook, we have:

variable | value | comment
---------|-------|:--------
method   | `GET` | Given in initial specification
protocol | `https` | Given in initial specification
location | `graph.facebook.com` | Given in initial specification
resource  | `/v2.12/oauth/access_token?` | Given in initial specification
client_id | `creds['facebook']['appid']` | Based on App registration and entering info in `creds.json`
client_secret | `creds['facebook']['appsecret']` | Based on App registration and entering info in `creds.json`
code | `creds['facebook']['code']` | Entered into creds.json based on Line 6 of OAuth Dance
state | `creds['facebook']['state']` | Entered into creds.json based on Line 6 of OAuth Dance

In [52]:
urlquery = {}
urlquery['client_id'] = creds['facebook']['appid']
urlquery['client_secret'] = creds['facebook']['appsecret']
urlquery['code'] = 'AQA73Rgor4IhQCoywILSuBdS8AtU4F1faQGobQBAq-TrcVlyHVwnMDkauKAOe5X1PfJHVpx3qAU0OSXT647cthLfhVywJDw066F2un-QEig_XvAy-c7cwhT23E6X82KLO-6wxJ2ahjTkxdthL4qtO-DJJ2TMMIC40_2Xg_XxCGj0jQXNmDzFhJCGj96_BqrtkIxbwnOAjsiJTM5F54-IExNTRhn2LwA480qFk6LL9RPW6wWGoCeveZkivHjPkauOQSc5PMKFPteslMXAo7xM5HpvaMn_g7IFRrGJlX_8BgcRJp265kA1OfvPqoJC9odj-dlB1HPIcl3Iy5cYyNzejgvG'

#creds['facebook']['code']
urlquery['redirect_uri'] = creds['facebook']['redirect_uri']
urlquery['state'] = creds['facebook']['state']
urlquery

{'client_id': '1532167366905914',
 'client_secret': '8cfdcfe4fb92e6af2119a6eff9325a36',
 'code': 'AQA73Rgor4IhQCoywILSuBdS8AtU4F1faQGobQBAq-TrcVlyHVwnMDkauKAOe5X1PfJHVpx3qAU0OSXT647cthLfhVywJDw066F2un-QEig_XvAy-c7cwhT23E6X82KLO-6wxJ2ahjTkxdthL4qtO-DJJ2TMMIC40_2Xg_XxCGj0jQXNmDzFhJCGj96_BqrtkIxbwnOAjsiJTM5F54-IExNTRhn2LwA480qFk6LL9RPW6wWGoCeveZkivHjPkauOQSc5PMKFPteslMXAo7xM5HpvaMn_g7IFRrGJlX_8BgcRJp265kA1OfvPqoJC9odj-dlB1HPIcl3Iy5cYyNzejgvG',
 'redirect_uri': 'https://localhost/callback',
 'state': '1234567890'}

In [53]:
def TokenGetParams(location, resource, pdict):
    '''
    This function takes three parameters: a location, a resource and a dictionary, 
    construct and issue an HTTP request using the parameters specified in the dictionary 
    to the URL from the location and resource using the GET HTTP method. It then checks 
    the content type, and obtain a string conisting of the body of the returned 
    data resource. If the content type is JSON, the function will return a Python data
    structure.
    
    Return: 
        - If the content type is JSON: a Python data structure.
        - If not: A text string with the data resource
    '''
    
    accessurl = buildBaseURL(location, auth_resource, protocol='https')
    headerD = {'Accept': 'application/json'}
    resp = requests.get(accessurl, headers=headerD, params=urlquery)
    if resp.status_code != 200:
        return None
    else:
        return resp.text

In [54]:
location = "graph.facebook.com"
auth_resource = "/v2.12/oauth/access_token?"
token = TokenGetParams(location, auth_resource, urlquery)
print(token)

https://graph.facebook.com/v2.12/oauth/access_token?
<Response [200]>
{"access_token":"EAAVxf4OkEDoBABmKWxShpP2EK0Q7rW0fZAvaPxGj0id1fh8wrVwQ4E9T8ZBFgnDzraiiK0oK1qfmvkWXH83mHWkdyvIfGCIu8zstosbliBPKLDS0aUpWJe3jZA4e8rVwxZBwNYDojZAAvSveFaNVQLZCavj28YVOEqp1daEPEeLAZDZD","token_type":"bearer","expires_in":5182364}


## C. Discussion

- At this point, we have obtained a single user token for myself. We initially planned to exchange this short-lived token for a long-lived one. However, we realized that each token could last for about 60 days (see the `expires_in` parameter above.
- Initially, Christien also authorized the app to read his Facebook data (the app requested to read `public_profile`, `user_likes`, `user_photos` and `user_posts`. However, as Facebook is changing its data policy, the app needs to submit a review in order to read any other type of data about the users (rather than the default `id` and `name`). Please see the following screenshot for more information on the types of App that needed to be reviewed by Facebook
https://developers.facebook.com/docs/apps/review

<img src="AppReview.png" width="750">